In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH11-Retriever")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH11-Retriever


In [3]:
# 문서를 예쁘게 출력하기 위한 도우미 함수
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"문서 {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

# 기본 Retriever 설정

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

# TextLoader를 사용하여 "appendix-keywords.txt" 파일에서 문서를 로드
loader = TextLoader(file_path="./data/appendix-keywords.txt", encoding="utf-8")

# CharacterTextSplitter를 사용하여 문서를 청크 크기 300, 청크간 중복 0으로 분할
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
texts = loader.load_and_split(text_splitter)

# OpenAIEmbeddings를 사용하여 FAISS 벡터 저장소를 생성하고 검색기로 변환
retriever = FAISS.from_documents(texts, embedding=OpenAIEmbeddings()).as_retriever()

# 쿼리에 질문을 정의하고 관련 문서를 검색
docs = retriever.invoke("Semantic Search에 대해서 알려줘.")

# 검색된 문서를 출력
pretty_print_docs(docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding
----------------------------------------------------------------------------------------------------
문서 2:

정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색

Page Rank
----------------------------------------------------------------------------------------------------
문서 3:

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec
----------------------------------------------------------------------------------------------------
문서 4:

정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으로, 주로 검색 엔진 결과의 순위를 결정하는 데 사용됩니다. 이는 웹 페이지 간의 링크 구조를 분석하여 평가합니다.
예시: 구글 검색 엔진은 페이

# 맥락적 압축(ContextualCompression)


In [5]:
from langchain_teddynote.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# LLM을 사용하여 문서 압축기 생성
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever,
)

pretty_print_docs(retriever.invoke("Semantic Search에 대해서 알려줘."))

print("=" * 30)
print("=" * 10, " LLMChainExtractor 적용 후 ", "="*10)

compressed_docs = (
    compression_retriever.invoke("Semantic Search에 해새서 알려줘.")
)
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding
----------------------------------------------------------------------------------------------------
문서 2:

정의: 키워드 검색은 사용자가 입력한 키워드를 기반으로 정보를 찾는 과정입니다. 이는 대부분의 검색 엔진과 데이터베이스 시스템에서 기본적인 검색 방식으로 사용됩니다.
예시: 사용자가 "커피숍 서울"이라고 검색하면, 관련된 커피숍 목록을 반환합니다.
연관키워드: 검색 엔진, 데이터 검색, 정보 검색

Page Rank
----------------------------------------------------------------------------------------------------
문서 3:

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec
----------------------------------------------------------------------------------------------------
문서 4:

정의: 페이지 랭크는 웹 페이지의 중요도를 평가하는 알고리즘으로, 주로 검색 엔진 결과의 순위를 결정하는 데 사용됩니다. 이는 웹 페이지 간의 링크 구조를 분석하여 평가합니다.
예시: 구글 검색 엔진은 페이

# LLM 을 활용한 문서 필터링

In [6]:
from langchain_teddynote.document_compressors import LLMChainFilter

# LLM을 사용하여 LLMChainFilter 객체를 생성
_filter = LLMChainFilter.from_llm(llm)

compression_retriever = ContextualCompressionRetriever(
    # LLMChainFilter와 retriever를 사용하여 ContextualCompressionRetriever객체를 생성
    base_compressor=_filter,
    base_retriever=retriever,
)

compressed_docs = compression_retriever.invoke("Semantic Search에 대해서 알려줘.")
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding


In [7]:
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

# 유사도  임계값이 0.86인 embeddingsFilter 객체를 생성
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.86)

# 기본 압축기로 embeddings_filter를, 기본검색기로 retriever를 사용하여 ContextualCompressionRetriever 객체를 생성
compression_retriever = ContextualCompressionRetriever(
    base_retriever=retriever,
    base_compressor=embeddings_filter,
)

# ContextualCompressionRetriever 객체를 사용하여 관련 문서를 검색
compressed_docs = compression_retriever.invoke("Semantic Search에 대해서 알려줘.")
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding


# 파이프라인 생성(압축기+문서 변환기)

In [8]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_text_splitters import CharacterTextSplitter

# 문자 기반 텍스트 분할기를 생성, 청크 크기는 300, 청크 간 중복은 0으로 설정
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

# 임베딩을 사용하여 중복 필터를 생성
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)

# 임베딩을 사용하여 관련성 필터를 생성, 유사도 임계값을 0.86으로 설정
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.86)

pipeline_compressor = DocumentCompressorPipeline(
    # 문서 압축 파이프라인을 생성하고, 분할기, 중복 필터, 관련성 필터, LLMChainExtractor를 변환기로 설정
    transformers=[
        splitter,
        redundant_filter,
        relevant_filter,
        LLMChainExtractor.from_llm(llm)
    ]
)

In [9]:
compression_retriever = ContextualCompressionRetriever(
    base_retriever=retriever,
    base_compressor=pipeline_compressor,
)

compressed_docs = compression_retriever.invoke("Semantic Search에 대해서 알려줘")
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
